In [1]:
#import urllib.request

In [2]:
#page = urllib.request.urlopen("https://en.wikipedia.org/wiki/List_of_state_and_union_territory_capitals_in_India")

In [1]:
import requests
from bs4 import BeautifulSoup

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Cafari/537.36'}
my_url = 'https://www.hltv.org/betting/money'

page = requests.get(my_url, headers=headers)
page_html = page.text

soup = BeautifulSoup(page_html,"html.parser")

print(soup.prettify())

<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" id="metaViewport" name="viewport"/>
  <meta content="1004164229" property="fb:admins"/>
  <meta content="249997999009" property="fb:pages"/>
  <meta content="1460388157605817" property="fb:app_id"/>
  <meta content="DcypRFLQvgYQL5Acx7feoGWbblSsmKv6HpPI7mM_1uw" name="google-site-verification"/>
  <link href="/img/static/favicon/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
  <link href="/img/static/favicon/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
  <link href="/img/static/favicon/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
  <link href="/img/static/favicon/manifest.json" rel="manifest"/>
  <link color="#5bbad5" href="/img/static/favicon/safari-pinned-tab.svg" rel="mask-icon"/>
  <meta content="#ffffff" name="theme-color"/>
  <link href="https://fonts.googleapis.com/css?family=Open+Sans:400,400i,700,700i|Oswald:

In [2]:
tables = soup.find_all('table', class_="bookmakerMatch")

In [3]:
columns = ["match", "url", "team1"]

data = []
c = 0
for table in tables:
        trs = table.find_all('tr')
        data_entry = []
        for tr in trs: # Every two tr is a match (one team per tr).
            
            match = tr.find_all('a', class_="a-reset")[0].get('href')
            
            teamname = tr.find_all('img', class_="team-logo")
            tds = tr.find_all('td', class_="betting-list-odds")
            #print("Odds len " + str(len(tds)))
            if (c==0):
                [columns.append(td.get("class")[-1].split('-')[-1]+ "-W") for td in tds]
                columns.append("team2")
                [columns.append(td.get("class")[-1].split('-')[-1]+ "-L") for td in tds]
            links = [td.find_all('a') for td in tds]
            odds = []
            for link in links:
                if len(link)>0:
                    odds.append(link[0].string)
                else:
                    odds.append("1.0")
            
            #print(tds[0].find_all('a')[0].string)
            if c % 2 == 0:
                # MatchId
                data_entry.append(match.split('/')[2])
                data_entry.append(match)
            
            data_entry.append(teamname[0].get('title')) #TeamName
            [data_entry.append(odd) for odd in odds] #Odds for team
            
            if c % 2 == 0:
                data.append(data_entry)
            c = c + 1
            
print(columns)
print(data[0])
print(len(columns))
print(len(data[0]))

['match', 'url', 'team1', 'betway-W', 'ggbet-W', 'thunderpick-W', 'lootbet-W', 'bet365-W', '1xbet-W', 'hidden-W', 'pinnacle-W', 'buff88-W', 'unibet-W', 'hidden-W', 'parimatch-W', 'skrilla-W', 'unikrn-W', 'rivalry-W', 'team2', 'betway-L', 'ggbet-L', 'thunderpick-L', 'lootbet-L', 'bet365-L', '1xbet-L', 'hidden-L', 'pinnacle-L', 'buff88-L', 'unibet-L', 'hidden-L', 'parimatch-L', 'skrilla-L', 'unikrn-L', 'rivalry-L']
['2331344', '/matches/2331344/pain-fe-vs-orgless-fe-intel-challenge-katowice-2019', 'paiN fe', '1.0', '1.08', '1.57', '1.0', '1.53', '1.13', '1.13', '1.0', '1.52', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0', 'Orgless fe', '1.0', '6.99', '2.29', '1.0', '2.37', '5.70', '5.70', '1.0', '2.37', '1.0', '1.0', '1.0', '1.0', '1.0', '1.0']
34
34


In [4]:
import pandas as pd

columns

In [5]:
df = pd.DataFrame(data=data, columns=columns)

In [6]:
columns.append("bestOf")


In [7]:
# Gets all bets of from match
for data_entry in data:
    url = "https://www.hltv.org" + data_entry[1]
    page = requests.get(url, headers=headers)
    page_html = page.text
    
    #print(url)
    soup = BeautifulSoup(page_html,"html.parser")
    #print(soup.prettify())
    div = soup.find_all('div', class_=["padding", "preformatted-text"])[0]
    bestOf = div.string.split('*')[0]
    bestOf = bestOf.replace(" (LAN)", '').replace(" (Online)", '').replace("\n", '').strip()
    team1 = div = soup.find_all('div', class_=["team1-gradient"])[0].a["href"]
    team2 = div = soup.find_all('div', class_=["team2-gradient"])[0].a["href"]
    print(url + ": " + bestOf + " (" + team1 + " vs " + team2)
    

https://www.hltv.org/matches/2331344/pain-fe-vs-orgless-fe-intel-challenge-katowice-2019: Best of 3 (/team/9871/pain-fe vs /team/7670/orgless-fe
https://www.hltv.org/matches/2331269/forze-vs-alternate-attax-esea-mdl-season-30-europe: Best of 1 (/team/8135/forze vs /team/4501/alternate-attax
https://www.hltv.org/matches/2331346/crowns-fe-vs-carnage-fe-intel-challenge-katowice-2019: Best of 3 (/team/7478/crowns-fe vs /team/9646/carnage-fe
https://www.hltv.org/matches/2331075/natus-vincere-vs-faze-iem-katowice-2019: Best of 3 (/team/4608/natus-vincere vs /team/6667/faze
https://www.hltv.org/matches/2331270/epsilon-vs-expert-esea-mdl-season-30-europe: Best of 1 (/team/4688/epsilon vs /team/9393/expert
https://www.hltv.org/matches/2331271/valiance-vs-ldlc-esea-mdl-season-30-europe: Best of 1 (/team/8481/valiance vs /team/4674/ldlc
https://www.hltv.org/matches/2331332/forze-vs-volgare-noxfire-league-season-2: Best of 1 (/team/8135/forze vs /team/5579/volgare
https://www.hltv.org/matches/2331

In [8]:
df.to_csv("data.csv", "a")

In [9]:
df.head()

,match,url,team1,betway-W,ggbet-W,thunderpick-W,lootbet-W,bet365-W,1xbet-W,hidden-W,...,1xbet-L,hidden-L,pinnacle-L,buff88-L,unibet-L,hidden-L,parimatch-L,skrilla-L,unikrn-L,rivalry-L
0,2331344,/matches/2331344/pain-fe-vs-orgless-fe-intel-c...,paiN fe,1.0,1.08,1.57,1.0,1.53,1.13,1.13,...,5.70,5.70,1.0,2.37,1.0,1.0,1.0,1.0,1.0,1.0
1,2331269,/matches/2331269/forze-vs-alternate-attax-esea...,forZe,1.09,1.11,1.45,1.10,1.44,1.16,1.16,...,5.00,5.00,6.32,6.44,3.00,2.88,6.25,1.0,6.50,2.73
2,2331346,/matches/2331346/crowns-fe-vs-carnage-fe-intel...,Crowns fe,1.0,1.04,1.81,1.0,1.66,1.0,1.0,...,1.0,1.0,1.0,2.35,1.0,1.0,1.0,1.0,1.0,1.0
3,2331075,/matches/2331075/natus-vincere-vs-faze-iem-kat...,Natus Vincere,1.50,1.52,1.67,1.54,1.53,1.62,1.62,...,2.26,2.26,2.13,2.24,2.35,2.38,2.53,2.47,2.37,2.27
4,2331270,/matches/2331270/epsilon-vs-expert-esea-mdl-se...,Epsilon,1.40,1.45,1.53,1.45,1.40,1.45,1.45,...,2.62,2.62,2.45,2.59,2.70,2.66,2.53,1.0,2.62,2.63
